In [22]:
# Import libraries
import numpy as np
import os , csv
from os import listdir
import matplotlib.pyplot as plt
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.interpolate import interp1d
from sklearn.decomposition import TruncatedSVD
%matplotlib inline
import matplotlib.pyplot as plt
import DataPreprocessing as spike_process



In [23]:
# Load desired data from 1 session 1 animal
# Note that there are 340 trials in 1 session
# (For more info https://github.com/nsteinme/steinmetz-et-al-2019/wiki/data-files)

data_path = '/Users/xinweichia/Documents/connected_lizards/Steinmetz_dataset/Richards_2017-10-31'

trials_intervals = np.load(data_path+'/'+'trials.intervals.npy') # in seconds
spike_times = np.load(data_path+'/'+'spikes.times.npy') * 1000 # Unbinned spike times in ms
trials_gocue_times = np.load(data_path+'/'+'trials.goCue_times.npy') 
trials_response_choice = np.load(data_path+'/'+'trials.response_choice.npy') # -1 left, 1, right, 0 no response
spontaneous_intervals = np.load(data_path+'/'+'spontaneous.intervals.npy')
trials_response_time = np.load(data_path+'/'+'trials.response_times.npy')
spike_clusters = np.load(data_path+'/'+'spikes.clusters.npy')
site_positions = np.load(data_path+'/'+'channels.sitePositions.npy')
clusters_depths = np.load(data_path+'/'+'clusters.depths.npy')
clusters_annotation = np.load(data_path+'/'+'clusters._phy_annotation.npy')
channel_sites = np.load(data_path+'/'+'channels.site.npy')
channels_brainlocation = pd.read_csv(data_path+'/'+'channels.brainLocation.tsv', sep='\t')
clusters_probes = np.load(data_path+'/'+'clusters.probes.npy')
channels_probe = np.load(data_path+'/'+'channels.probe.npy')
trials_visual_time = np.load(data_path+'/'+'trials.visualStim_times.npy')
trial_feedback_time = np.load(data_path+'/'+'trials.feedback_times.npy')
trial_feedback_type = np.load(data_path+'/'+'trials.feedbackType.npy')
trial_response_time = np.load(data_path+'/'+'trials.response_times.npy')

visual_times = trials_visual_time
# Behaviour data

wheel_movement = np.load(data_path+'/'+'wheelMoves.type.npy')
wheel_intervals = np.load(data_path+'/'+'wheelMoves.intervals.npy')


In [24]:
dt = 1/100
dT = 2.5
T0 = .5

# Get neural activity data
good_cells, brain_regions ,br = spike_process.get_good_cells(data_path) # Get brain regions
spike_time_binned, spike_time_cells, cell_spikes_max = spike_process.bin_spikes(spike_times,spike_clusters,clusters_annotation,10)
#spike_time_binned_trial, spike_time_binned_trial_response = spike_process.sort_cells_trials(spike_time_binned, spike_time_cells, trials_intervals,trials_visual_time, epoch_duration = 2000,10)
spike_time_binned_trial, pre_stim_spike_time_binned_trial, post_stim_spike_time_binned_trial, post_go_cue_spike_time_binned_trial, post_go_cue_spike_time_binned_trial = spike_process.sort_cells_behaviour_trials(spike_time_binned,spike_time_cells,10)
left_spike_time_response, right_spike_time_response, no_response_spike_time_response = spike_process.sort_cells_trial_types(post_stim_spike_time_binned_trial,trials_intervals,spike_time_cells,trials_response_choice)

# Midbrain cells
midbrain_left_spike = spike_process.sort_cells_brain_regions(left_spike_time_response, 'SCs','SCm','MRN','APN','PAG','ZI',brain_regions,clusters_annotation)
midbrain_right_spike = spike_process.sort_cells_brain_regions(right_spike_time_response, 'SCs','SCm','MRN','APN','PAG','ZI',brain_regions,clusters_annotation)
midbrain_no_response_spike = spike_process.sort_cells_brain_regions(no_response_spike_time_response, 'SCs','SCm','MRN','APN','PAG','ZI',brain_regions,clusters_annotation)
midbrain_concat_left_spike_times = spike_process.concat_trials_2_timeseries(midbrain_left_spike)
midbrain_concat_right_spike_times = spike_process.concat_trials_2_timeseries(midbrain_right_spike)
midbrain_concat_no_response_spike_times = spike_process.concat_trials_2_timeseries(midbrain_no_response_spike)

# Forebrain cells
forebrain_left_spike = spike_process.sort_cells_brain_regions(left_spike_time_response, 'MOs','PL','ILA','ORB','MOp','SSp',brain_regions,clusters_annotation)
forebrain_right_spike = spike_process.sort_cells_brain_regions(right_spike_time_response, 'MOs','PL','ILA','ORB','MOp','SSp',brain_regions,clusters_annotation)
forebrain_no_response_spike = spike_process.sort_cells_brain_regions(no_response_spike_time_response, 'MOs','PL','ILA','ORB','MOp','SSp',brain_regions,clusters_annotation)
forebrain_concat_left_spike_times = spike_process.concat_trials_2_timeseries(forebrain_left_spike)
forebrain_concat_right_spike_times = spike_process.concat_trials_2_timeseries(forebrain_right_spike)
forebrain_concat_no_response_spike_times = spike_process.concat_trials_2_timeseries(forebrain_no_response_spike)

# Get behaviour data
wheel, wheel_times = spike_process.get_wheel(data_path)
wheel_position = spike_process.wpsth(wheel, wheel_times,visual_times-T0, dT, dt)
left_concat_wheel_position, right_concat_wheel_position, no_response_concat_wheel_position = spike_process.concat_behaviour_2_timeseries(wheel_position, epoch_duration = 400 ,bin_size = 10)



TypeError: sort_cells_behaviour_trials() takes from 1 to 2 positional arguments but 3 were given

In [ ]:
print(forebrain_concat_left_spike_times.shape)
print(midbrain_left_spike.shape)

In [ ]:
good_cells, brain_regions ,br = spike_process.get_good_cells(data_path) # Get brain regions
valid_brain_region_boo = (np.reshape( (clusters_annotation >= 2),(-1) ) & np.reshape( (brain_regions == 'SCm'),(-1) ))

valid_brain_region_boo.shape

In [ ]:

good_cells, brain_regions ,br = spike_process.get_good_cells(data_path) # Get brain regions
def sort_cells_brain_regions(spike_time_response, *input_region):
    
    regional_spike_time_response = np.empty(len(input_region), dtype=object)
    for i,region in enumerate(input_region):
        # Get brain regions that correponds to the desired region
        valid_brain_region_boo = (np.reshape( (clusters_annotation >= 2),(-1) ) & np.reshape( (brain_regions == region),(-1) ))
        # Index the spike time to get spikes from desired regions
        regional_spike_time_response[i] = spike_time_response[valid_brain_region_boo]
    
    # Merge spikes across all regions
    merged_region_spikes = []
    for i in np.arange(len(regional_spike_time_response)):
        merged_region_spikes = np.append(merged_region_spikes, regional_spike_time_response[i])
    
    # Return spike time sorted into regions and merged across all regions
    # Indexing: regional_spike_times[cell_num][trial_num][# of bins]
    return merged_region_spikes

mid_brain_circuits=['SCs','SCm','MRN','APN','PAG','ZI']
frontal_circuits=['MOs','PL','ILA','ORB','MOp','SSp']



In [ ]:
print(len(spike_time_cells))
print(len(brain_regions))


In [ ]:

print(forebrain_concat_left_spike_times.shape)
print(midbrain_concat_left_spike_times.shape)

print(data_path)

In [ ]:
# Perform some quick visualisation on the spike and behaviour data

plt.figure(0)
cell_num = 42
plt.plot(midbrain_concat_left_spike_times[cell_num])
plt.title('Spikes for 1 cell across session (10ms bin)')
plt.show

plt.figure(1)
plt.plot(left_concat_wheel_position)
plt.title('Wheel position across session')
plt.show



In [ ]:
# Perform regression analysis

corr_coeff = np.array([])
for cell_num in np.arange(len(midbrain_concat_left_spike_times)):
    corr_coeff = np.append(corr_coeff,np.corrcoef(midbrain_concat_left_spike_times[cell_num], left_concat_wheel_position)[0,1])

plt.figure(0)
plt.hist(corr_coeff,bins = 20)
plt.title('Midbrain Correlation Coeff with Wheel')
plt.xlim((-0.08, 0.08))
plt.show

corr_coeff = np.array([])
for cell_num in np.arange(len(forebrain_concat_left_spike_times)):
    corr_coeff = np.append(corr_coeff,np.corrcoef(forebrain_concat_left_spike_times[cell_num], left_concat_wheel_position)[0,1])

plt.figure(1)
plt.hist(corr_coeff,bins = 20)
plt.title('Forebrain Correlation Coeff with Wheel')
plt.xlim((-0.08, 0.08))
plt.show

corr_coeff = np.array([])
for cell_num in np.arange(len(forebrain_concat_left_spike_times)):
    corr_coeff = np.append(corr_coeff,np.corrcoef(forebrain_concat_left_spike_times[cell_num], left_concat_wheel_position)[0,1])

plt.figure(1)
plt.hist(corr_coeff,bins = 20)
plt.title('Forebrain Correlation Coeff with Wheel')
plt.xlim((-0.08, 0.08))
plt.show



In [ ]:
print(corr_coeff)